# Speech to Text

In [ ]:
!pip install SpeechRecognition pyttsx3 AudioSegment gTTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73462 sha256=a17f85b7f550378eeebc3282834770f8d7759b79c628de25b52248f0ddf3885a
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad


In [ ]:
import speech_recognition as sr
from gtts import gTTS

def voice_to_text(audio_file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data)

    return text

def text_to_speech(text, output_file_path):
    tts = gTTS(text)
    tts.save(output_file_path)

def text_to_binary(text, output_file_path):
    binary_data = ' '.join(format(ord(char), '08b') for char in text)

    # Save binary data to a file
    with open(output_file_path, 'w') as binary_file:
        binary_file.write(binary_data)

def binary_to_text(binary_data):
    binary_values = binary_data.split(' ')
    text = ''.join([chr(int(b, 2)) for b in binary_values])
    return text

def main_menu():
    print("1. Transcribe Voice to Text")
    print("2. Text to Speech")
    print("3. Exit")

def main():
    while True:
        main_menu()
        choice = input("Enter your choice (1, 2, or 3): ")

        if choice == '1':
            audio_file_path = input("Enter the path of the audio file: ")
            transcribed_text = voice_to_text(audio_file_path)
            print("\nTranscribed Text:")
            print(transcribed_text)

            binary_output_path = input("Enter the path to save the binary representation (including file extension): ")
            text_to_binary(transcribed_text, binary_output_path)
            print(f"Binary representation saved to: {binary_output_path}\n")

        elif choice == '2':
            binary_text = input("Enter the text to convert to speech: ")
            text = binary_to_text(binary_text)
            output_file_path = input("Enter the path to save the speech (including file extension): ")
            text_to_speech(text, output_file_path)
            print(f"Speech saved to: {output_file_path}\n")

        elif choice == '3':
            print("Exiting program. Goodbye!")
            break

        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()


1. Transcribe Voice to Text
2. Text to Speech
3. Exit
Enter your choice (1, 2, or 3): 1
Enter the path of the audio file: plaintext.wav

Transcribed Text:
hello good night everyone
Enter the path to save the binary representation (including file extension): hasil_encoding.txt
Binary representation saved to: hasil_encoding.txt

1. Transcribe Voice to Text
2. Text to Speech
3. Exit
Enter your choice (1, 2, or 3): 3
Exiting program. Goodbye!


# Encrypt & Decrypt dengan RSA

In [ ]:
import random
import math

# Fungsi untuk mengecek apakah suatu bilangan merupakan bilangan prima
def is_prime(number):
    if number < 2:
        return False
    for i in range(2, number // 2 + 1):
        if number % i == 0:
            return False
    return True

# Fungsi untuk mengecek apakah suatu bilangan merupakan bilangan prima
def get_prime_input(message):
    while True:
        try:
            user_input = int(input(message))
            if is_prime(user_input):
                return user_input
            else:
                print("Input is not a prime number. Try again.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

# Fungsi untuk mencari nilai d (private key) dari nilai e (public key) dan totient n
def mod_inverse(e, totient):
    for d in range(3, totient):
        if (d * e) % totient == 1:
            return d
    raise ValueError('No mod inverse for e: %d, totient: %d' % (e, totient))

def make_key():
    p = get_prime_input("Enter a prime number (p): ")
    q = get_prime_input("Enter a prime number (q): ")

    # p dan q tidak boleh sama
    while p == q:
        print("p and q cannot be the same number. Try again.")
        p = get_prime_input("Enter a prime number (p): ")
        q = get_prime_input("Enter a prime number (q): ")

    # nilai n adalah hasil perkalian p dan q dan boleh dibagikan
    n = p * q
    # nilai totient n adalah hasil perkalian p-1 dan q-1 tidak boleh dibagikan karena untuk mencari nilai d
    totient_n = (p - 1) * (q - 1)

    # nilai e adalah nilai yang tidak boleh dibagikan dan harus lebih besar dari 2 dan lebih kecil dari Pembagi persekutuan terbesar / GCD
    e = int(input(f"Enter a public exponent (e) such that 2 < e < {totient_n} and gcd(e, {totient_n}) = 1: "))

    while not (1 < e < totient_n and math.gcd(e, totient_n) == 1):
        print(f"Invalid value for e. Try again.")
        e = int(input(f"Enter a public exponent (e) such that 2 < e < {totient_n} and gcd(e, {totient_n}) = 1: "))

    d = mod_inverse(e, totient_n)

    print("Public key (e, n): ", e, n)
    print("Private key (d, n): ", d, n)


def encryption_menu():
    n = int(input("Masukan nilai n: "))
    e = int(input("Masukan kunci public e: "))
    input_path = input("Masukkan path file untuk dienkripsi (contoh: hasil_encoding.txt): ")
    process_path = input("Masukkan path untuk menyimpan process enkripsi (contoh: process_enkripsi.txt): ")
    output_path = input("Masukkan path untuk menyimpan hasil enkripsi (contoh: hasil_enkripsi.txt): ")

    with open(input_path, 'r') as file:
        # Baca seluruh konten file
        message = file.read()

    # Mengubah pesan menjadi nilai ASCII dan enkripsi
    ciphertext = []
    max_digit = 0
    values = []

    # Open the file in append mode
    with open(process_path, "a") as base64_file:
        # Iterate through each character in the message
        for i in range(0, len(message)):
            block = message[i]
            encoded_block = ord(block)  # Convert character to ASCII value
            encrypted_block = pow(encoded_block, e, n)

            # Append the value to the list
            values.append(encrypted_block)

            # Write the details of the current block to the file
            base64_file.write(f"Block '{block}' is {encoded_block}, {encoded_block}^{e} mod {n} is {encrypted_block}\n")

            current_digit = len(str(encrypted_block))
            max_digit = max(max_digit, current_digit)
            ciphertext.append(encrypted_block)

    ciphertext_str = ''.join([str(block).zfill(max_digit) for block in ciphertext])

     # Simpan hasil enkripsi ke dalam file teks
    with open(output_path, "w") as base64_file:
        base64_file.write(ciphertext_str)
        print("Sukses! Hasil enkripsi tersimpan")

    return max_digit


def decryption_menu(max_digit):
    n = int(input("Masukan nilai n: "))
    d = int(input("Masukan kunci private d: "))
    input_path = input("Masukkan path file untuk didekripsi (contoh: hasil_enkripsi.txt): ")
    process_path = input("Masukkan path untuk menyimpan process dekripsi (contoh process_dekripsi.txt): ")
    output_path = input("Masukkan path untuk menyimpan hasil dekripsi (contoh: hasil_dekripsi.txt): ")

    with open(input_path, 'r') as file:
        # Baca seluruh konten file
        isi_file = file.read()

    ciphertext_str = isi_file

    # Split ciphertext into blocks of size max_digit
    ciphertext_blocks = [int(ciphertext_str[i:i + max_digit]) for i in range(0, len(ciphertext_str), max_digit)]

    decrypted_message = ""

    values = []

    # Open the file in append mode
    with open(process_path, "a") as base64_file:
        # Iterate through each block in the ciphertext
        for block in ciphertext_blocks:
            decrypted_block = pow(block, d, n)
            decrypted_char = chr(decrypted_block)  # Convert ASCII value to character

            # Append the value to the list
            values.append(decrypted_block)

            # Write the details of the current block to the file
            base64_file.write(f"{block}^{d} mod {n} is {decrypted_block} which is '{decrypted_char}'\n")

            decrypted_message += decrypted_char

    # Simpan hasil dekripsi ke dalam file teks
    with open(output_path, "w") as base64_file:
        base64_file.write(decrypted_message)
        print("Sukses! Hasil dekripsi tersimpan")

# Fungsi untuk menghitung totient n dari dua bilangan prima p dan q
def calculate_totient(p, q):
    return (p - 1) * (q - 1)

# Fungsi untuk mencari bilangan prima yang relatif prima terhadap totient n
def generate_coprime(num_digits, totient_n):
    while True:
        random_prime = generate_random_prime(num_digits)
        if math.gcd(random_prime, totient_n) == 1:
            return random_prime


# Fungsi untuk mengenerate bilangan prima dengan jumlah digit yang diinginkan
def generate_random_prime(num_digits):
    min_value = 10 ** (num_digits - 1)
    max_value = (10 ** num_digits) - 1

    while True:
        random_prime = random.randint(min_value, max_value)
        if is_prime(random_prime):
            return random_prime

# Fungsi untuk menampilkan menu bilangan prima custom
def custom_prime_menu():
    num_digits = input("Masukkan jumlah digit bilangan prima yang diinginkan: ")
    # generate bilangan prima dengan jumlah digit yang diinginkan
    prime1 = generate_random_prime(int(num_digits))
    prime2 = generate_random_prime(int(num_digits))

    if prime1 is not None and prime2 is not None:
        print(f"Bilangan prima dengan {num_digits} digit (p): {prime1}")
        print(f"Bilangan prima dengan {num_digits} digit (q): {prime2}")
        totient_n = calculate_totient(prime1, prime2)
        e = generate_coprime(int(num_digits), totient_n)

        print(f"Totient n: {totient_n}")
        print(f"Bilangan prima relatif prima terhadap totient n (e): {e}")
    else  :
        print(f"Tidak ada bilangan prima dengan {num_digits} digit")


if __name__ == "__main__":
    max_digit = 3  # Initialize max_digit outside the loop
    while True:
        print("\nMenu:")
        print("1. Make keys")
        print("2. Encryption")
        print("3. Decryption")
        print("4. Custom Prime")
        print("5. Exit")

        choice = input("Enter your choice (1, 2, 3, 4, or 5): ")
        if choice == '1':
            make_key()  # Capture max_digit from make_key
        elif choice == '2':
            max_digit = encryption_menu()
        elif choice == '3':
            decryption_menu(max_digit)  # Pass max_digit to decryption_menu
        elif choice == '4':
            custom_prime_menu()
        elif choice == '5':
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5")



Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Enter your choice (1, 2, 3, 4, or 5): 4
Masukkan jumlah digit bilangan prima yang diinginkan: 4
Bilangan prima dengan 4 digit (p): 6871
Bilangan prima dengan 4 digit (q): 4657
Totient n: 31986720
Bilangan prima relatif prima terhadap totient n (e): 1061

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Enter your choice (1, 2, 3, 4, or 5): 1
Enter a prime number (p): 6871
Enter a prime number (q): 4657
Enter a public exponent (e) such that 2 < e < 31986720 and gcd(e, 31986720) = 1: 1061
Public key (e, n):  1061 31998247
Private key (d, n):  31052141 31998247

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Enter your choice (1, 2, 3, 4, or 5): 2
Masukan nilai n: 31998247
Masukan kunci public e: 1061
Masukkan path file untuk dienkripsi (contoh: hasil_encoding.txt): hasil_encoding.txt
Masukkan path untuk menyimpan process enkripsi (contoh: process_enkripsi.txt): process_en

# Hasil Dekripsi Voice

In [ ]:
import speech_recognition as sr
from gtts import gTTS

def voice_to_text(audio_file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data)

    return text

def text_to_speech(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file:
        text = input_file.read()

    tts = gTTS(text)
    tts.save(output_file_path)

def text_to_binary(text, output_file_path):
    binary_data = ' '.join(format(ord(char), '08b') for char in text)

    # Save binary data to a file
    with open(output_file_path, 'w') as binary_file:
        binary_file.write(binary_data)

def binary_file_to_text(file_path):
    with open(file_path, 'r') as binary_file:
        binary_data = binary_file.read()
    text_data = binary_to_text(binary_data)
    return text_data

def main_menu():
    print("1. Transcribe Voice to Text")
    print("2. Text to Speech")
    print("3. Exit")

def main():
    while True:
        main_menu()
        choice = input("Enter your choice (1, 2, or 3): ")

        if choice == '1':
            audio_file_path = input("Enter the path of the audio file: ")
            transcribed_text = voice_to_text(audio_file_path)
            print("\nTranscribed Text:")
            print(transcribed_text)

            binary_output_path = input("Enter the path to save the binary representation (including file extension): ")
            text_to_binary(transcribed_text, binary_output_path)
            print(f"Binary representation saved to: {binary_output_path}\n")

        elif choice == '2':
            input_binary_file_path = input("Enter the path of the binary input file: ")
            output_file_path = input("Enter the path to save the speech (including file extension): ")

            # Convert binary data to text
            binary_text = binary_file_to_text(input_binary_file_path)

            # Save the text to a file
            with open(output_file_path, 'w') as text_file:
                text_file.write(binary_text)

            # Convert the text to speech
            text_to_speech(output_file_path, output_file_path)
            print(f"Speech saved to: {output_file_path}\n")

        elif choice == '3':
            print("Exiting program. Goodbye!")
            break

        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()


1. Transcribe Voice to Text
2. Text to Speech
3. Exit
Enter your choice (1, 2, or 3): 2
Enter the path of the binary input file: hasil_dekripsi.txt
Enter the path to save the speech (including file extension): hasil_dekrip_voice.wav
Speech saved to: hasil_dekrip_voice.wav

1. Transcribe Voice to Text
2. Text to Speech
3. Exit
Enter your choice (1, 2, or 3): 3
Exiting program. Goodbye!
